In [130]:
import pandas as pd
import re
from DataCollector import DataCollector
DATA_DIR = "./src/data"

In [131]:
collector = DataCollector(DATA_DIR)

In [132]:
from rdflib import Graph, URIRef, Literal, Namespace
from rdflib.namespace import RDF, XSD

In [133]:
# Criar o modelo (grafo RDF)
model = Graph()

# Definir namespaces 
base = Namespace("http://www.semanticweb.org/ericarfs/ontologies/2024/10/famouspaintings#")
xsd  = Namespace("http://www.w3.org/2001/XMLSchema#")
foaf = Namespace("http://xmlns.com/foaf/0.1/")
rdfs = Namespace("http://www.w3.org/2000/01/rdf-schema#")
rdf  = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")


# Vincular os namespaces ao grafo 
model.bind("", base)
model.bind("xsd", xsd)
model.bind("foaf", foaf)
model.bind("rdf", rdf)
model.bind("rdfs", rdfs)

In [134]:
def add_countries_to_model():
    countries = collector.get_countries()

    for country_name in countries:
        country_name_formated = re.sub(r"[^a-zA-Z0-9]", "", country_name)
        country = URIRef(base + country_name_formated)
        model.add((country, RDF.type, base.Country))
        model.add((country, foaf.name, Literal(country_name)))

In [135]:
def add_styles_to_model():
    artist_styles = collector.get_artist_styles()
    work_styles = collector.get_work_styles()
    styles = set(artist_styles + work_styles)

    for style_name in styles:
        style_name_formated = "Style_"+re.sub(r"[^a-zA-Z0-9]", "", style_name)
        style = URIRef(base + style_name_formated)
        model.add((style, RDF.type, base.Style))
        model.add((style, foaf.name, Literal(style_name)))

In [136]:
def add_subjects_to_model():
    subjects = collector.get_subjects()

    for subject_name in subjects:
        subject_name_formated = "Subject_"+re.sub(r"[^a-zA-Z0-9]", "", subject_name)
        subject = URIRef(base + subject_name_formated)
        model.add((subject, RDF.type, base.Subject))
        model.add((subject, foaf.name, Literal(subject_name)))

In [137]:
def add_nationalities_to_model():
    nationalities = collector.get_artist_nationalities()

    for nationality_name in nationalities:
        nationality_name_formated = re.sub(r"[^a-zA-Z0-9]", "", nationality_name)
        nationality = URIRef(base + nationality_name_formated)
        model.add((nationality, RDF.type, base.Nationality))
        model.add((nationality, foaf.name, Literal(nationality_name)))

In [138]:
def add_museums_to_model():
    data = collector.get_museums()
    tam = data.shape[0]
    
    for i in range (tam):
        museum_identifier = data.iloc[i]["identifier"]
        museum = URIRef(base + museum_identifier)
        model.add((museum, RDF.type, base.Museum))

        museum_name = str(data.iloc[i]["name"])
        model.add((museum, foaf.name, Literal(museum_name)))

        country_name = re.sub(r"[^a-zA-Z0-9]", "", str(data.iloc[i]["country"]))
        if country_name != "nan":
            model.add((museum, base.hasCountry, URIRef(base + country_name )))

        state = str(data.iloc[i]["state"])
        if state != "nan":
            model.add((museum, base.state, Literal(state)))
        
        city = str(data.iloc[i]["city"])
        if city != "nan":
            model.add((museum, base.city, Literal(city)))
        
        address = str(data.iloc[i]["address"])
        if address != "nan":
            model.add((museum, base.address, Literal(address)))
        
        postal = str(data.iloc[i]["postal"])
        if postal != "nan":
            model.add((museum, base.postal, Literal(postal)))
        
        phone = str(data.iloc[i]["phone"])
        if phone != "nan":
            model.add((museum, base.phone, Literal(phone)))
        
        url = str(data.iloc[i]["url"])
        if city != "nan":
            model.add((museum, base.url, Literal(url)))

In [139]:
def add_artists_to_model():
    data = collector.get_artists()
    tam = data.shape[0]
    
    for i in range (tam):
        artist_identifier = data.iloc[i]["identifier"]
        artist = URIRef(base + artist_identifier)
        model.add((artist, RDF.type, base.Artist))

        full_name = str(data.iloc[i]["full_name"])
        model.add((artist, foaf.name, Literal(full_name)))

        first_name = str(data.iloc[i]["first_name"])
        if first_name != "nan":
            model.add((artist, foaf.firstName, Literal(first_name)))

        last_name = str(data.iloc[i]["last_name"])
        if last_name != "nan":
            model.add((artist, foaf.lastName, Literal(last_name)))
        
        birth = str(data.iloc[i]["birth"])
        if birth != "nan":
            model.add((artist, base.birth, Literal(birth)))

        death = str(data.iloc[i]["death"])
        if death != "nan":
            model.add((artist, base.death, Literal(death)))
        
        nationality = re.sub(r"[^a-zA-Z0-9]", "", str(data.iloc[i]["nationality"]))
        if nationality != "nan":
            model.add((artist, base.hasNationality, URIRef(base + nationality )))
        
        style = "Style_"+re.sub(r"[^a-zA-Z0-9]", "", str(data.iloc[i]["style"]))
        if style != "nan":
            model.add((artist, base.hasStyle, URIRef(base + "Style_"+ style )))

In [140]:
def add_works_to_model():
    data = collector.get_works()
    tam = data.shape[0]

    for i in range (tam):
        work_identifier = data.iloc[i]["identifier"]
        work = URIRef(base + work_identifier)
        model.add((work, RDF.type, base.Work))

        name = str(data.iloc[i]["name"])
        model.add((work, foaf.name, Literal(name)))

        artist = collector.get_artist_by_id(data.iloc[i]["artist_id"])
        if artist is not None:
            model.add((work, base.wasPaintedBy, URIRef(base + artist )))
        
        style = "Style_" + re.sub(r"[^a-zA-Z0-9]", "", str(data.iloc[i]["style"]))
        if style != "nan":
            model.add((work, base.hasStyle, URIRef(base + "Style_"+ style )))
        
        subject = collector.get_subject_by_id(data.iloc[i]["work_id"])
        if subject is not None:
            model.add((work, base.hasSubject, URIRef(base + subject )))

        museum_id = data.iloc[i]["museum_id"]
        if museum_id != "nan":
            work_museum = collector.get_museum_by_id(museum_id)
            if work_museum is not None:
                model.add((work, base.isLocatedAt, URIRef(base + work_museum)))
        

In [141]:
add_nationalities_to_model()
add_subjects_to_model()
add_countries_to_model()
add_styles_to_model()
add_works_to_model()
add_museums_to_model()
add_artists_to_model()

In [142]:
# Serializar o grafo para um arquivo ou string
output_file = "output.ttl"  
with open(output_file, "w", encoding="utf-8") as fout:
    fout.write(model.serialize(format="turtle"))

In [143]:
from owlrl import DeductiveClosure, RDFS_Semantics

In [144]:
schema_file = "./famouspaintings.ttl"
data_file = "./output.ttl"

schema = Graph()
schema.parse(schema_file)  # Ler o esquema

data = Graph()
data.parse(data_file, format="turtle")  # Ler os dados 

inferred_model = Graph()
inferred_model += schema
inferred_model += data

DeductiveClosure(RDFS_Semantics).expand(inferred_model)  # Inferência RDFS

print("Modelo inferido:")
output_file2 = "output2.ttl"  
with open(output_file2, "w", encoding='utf-8') as fout:
    fout.write(inferred_model.serialize(format="turtle"))

Modelo inferido:


In [53]:
museums = collector.get_museums()


In [23]:
museums_name = museums["name"].tolist()

In [54]:
from geopy.geocoders import Nominatim
import random

def obter_lat_lon_nominatim(endereco):
    geolocator = Nominatim(user_agent="sua_aplicacao")
    localizacao = geolocator.geocode(endereco)
    if localizacao:
        return localizacao.latitude, localizacao.longitude
    else:
        print("Endereço não encontrado")
        return None, None

# Exemplo de uso
elemento_aleatorio = random.choice(museums_name)
endereco = random.choice(museums_name)
latitude, longitude = obter_lat_lon_nominatim(endereco)
print(f"Latitude: {latitude}, Longitude: {longitude}")

import folium

# create tuples representing our location
location = float(latitude), float(longitude)

# center the map 
center = (latitude, longitude)

# create a Folium map centred at the above location
m = folium.Map(location=center, zoom_start=50, width=800, height=400)

# add markers at the locations
folium.Marker(location, popup="The postcode brought me here").add_to(m)

# refer to the map to display it in Jupyter/Colab notebooks
m

Latitude: 38.89131155, Longitude: -77.01990447478894


In [54]:
# Pesquisando dados de artistas por inicio do nome
searchQuery = 'Ad'.lower()
qres = inferred_model.query(
    f"""
        PREFIX : <http://www.semanticweb.org/ericarfs/ontologies/2024/10/famouspaintings#>
        
        SELECT ?artistName ?birth ?death ?style ?nationality
        WHERE {{
            ?artist
                rdf:type :Artist ;
                foaf:name ?artistName ;
            .
            OPTIONAL {{?artist :birth ?birth .}}
            OPTIONAL {{?artist :death ?death .}}
            OPTIONAL {{?artist :hasStyle/foaf:name ?style .}}
            OPTIONAL {{?artist :hasNationality/foaf:name ?nationality .}}
            FILTER (STRSTARTS(LCASE(STR(?artistName)), "{searchQuery}"))
        }}
    """,
    initNs={
        'rdf': rdf,
        'foaf': foaf,
    }
)

for row in qres:
    print(' | '.join([e or 'NULL' for e in row]))

Adriaan De Lelie | 1755 | 1820 | Portraitist | Dutch
Adèle Romany | 1769 | 1846 | Portraitist | French
Adolph Ulrich Wertmuller | 1751 | 1811 | Portraitist | Swiss
Adriaen Thomasz. Key | 1544 | 1589 | Renaissance | Dutch
Adélaïde Labille-Guiard | 1749 | 1803 | Neoclassical | French


In [153]:
# Pesquisando dados de museus por inicio do nome
searchQuery = 'Mus'.lower()
qres = inferred_model.query(
    f"""
        PREFIX : <http://www.semanticweb.org/ericarfs/ontologies/2024/10/famouspaintings#>
        
        SELECT ?museumName ?country ?state ?city ?address ?postal ?phone ?url
        WHERE {{
            ?museum
                rdf:type :Museum ;
                foaf:name ?museumName ;
            .
            OPTIONAL {{?museum :hasCountry/foaf:name ?country .}}
            OPTIONAL {{?museum :state ?state .}}
            OPTIONAL {{?museum :city ?city .}}
            OPTIONAL {{?museum :address ?address .}}
            OPTIONAL {{?museum :postal ?postal .}}
            OPTIONAL {{?museum :phone ?phone .}}
            OPTIONAL {{?museum :url ?url .}}
            FILTER (STRSTARTS(LCASE(STR(?museumName)), "{searchQuery}"))
        }}
    """,
    initNs={
        'rdf': rdf,
        'foaf': foaf,
    }
)

for row in qres:
    print(' | '.join([e or 'NULL' for e in row]))

Musée Marmottan Monet | France | NULL | Paris | 2 Rue Louis Boilly | 75016 | +33 1 44 96 50 33 | https://www.marmottan.fr/en/
Museum of Fine Arts, Houston | USA | TX | Houston | 1001 Bissonnet St | 77005 | +1 713 639-7300 | https://www.mfah.org/
Museum of Fine Arts of Nancy | France | NULL | Nancy | 3 Pl. Stanislas | 54000 | +33 3 83 85 30 01 | https://useum.org/museum/Museum-of-Fine-Arts-of-Nancy
Musée d'Orsay | France | NULL | Paris | 1 Rue de la Légion d'Honneur | 75007 | +33 (0)1 40 49 48 14 | https://www.musee-orsay.fr/en
Museum of Grenoble | France | Grenoble | 38000 | 5 Pl. de Lavalette | NULL | 33 4 76 63 44 44 | https://www.museedegrenoble.fr/1986-the-museum-in-english.htm
Museum of Fine Arts Boston | USA | MA | Boston | 465 Huntington Ave | 2115 | 617-267-9300 | https://www.mfa.org/
Museum of Fine Arts | Switzerland | NULL | Bern | Hodlerstrasse 8 | 3011 | +41 31 328 09 44 | https://www.kunstmuseumbern.ch/en/
Musée des Beaux-Arts de Quimper | France | Quimper | 29000 | 40 Pl.

In [154]:
# Pesquisando dados de obras por inicio do nome
searchQuery = 'The Grand'.lower()
qres = inferred_model.query(
    f"""
        PREFIX : <http://www.semanticweb.org/ericarfs/ontologies/2024/10/famouspaintings#>
        
        SELECT ?workName ?artist ?style ?subject ?location
        WHERE {{
            ?work
                rdf:type :Work ;
                foaf:name ?workName ;
            .
            OPTIONAL {{?work :wasPaintedBy/foaf:name ?artist .}}
            OPTIONAL {{?work :hasStyle/foaf:name ?style .}}
            OPTIONAL {{?work :hasSubject/foaf:name ?subject .}}
            OPTIONAL {{?work :isLocatedAt/foaf:name ?location .}}
            FILTER (STRSTARTS(LCASE(STR(?workName)), "{searchQuery}"))
        }}
    """,
    initNs={
        'rdf': rdf,
        'foaf': foaf,
    }
)

for row in qres:
    print(' | '.join([e or 'NULL' for e in row]))

The Grand Canal in Venice from Palazzo Flangini to Campo San Marcuola | Canaletto | Rococo | Architectures | The J. Paul Getty Museum
The Grand Canal, Venice | Joseph M. W. Turner | Neo-Classicism | Rivers/Lakes | The Metropolitan Museum of Art
The Grand Canal in Venice with Palazzo Bembo | Francesco Guardi | Rococo | Architectures | The J. Paul Getty Museum
The Grand Canal Above the Rialto | Francesco Guardi | Rococo | Architectures | The Metropolitan Museum of Art
The Grand Canal Blue Venice | Édouard Manet | Impressionism | Rivers/Lakes | NULL
The Grand Canal from Palazzo Flangini to Campo San Marcuola | Canaletto | Rococo | Architectures | NULL
The Grand Bath at Bursa | Jean-Léon Gérôme | Orientalism | Nude | NULL
The Grand Canal in Venice | Albert Marquet | Post-Impressionism | Rivers/Lakes | NULL
The Grand Canyon of the Colorado | Thomas Moran | American Landscape | Landscape Art | NULL
The Grand White Eunuch | Jean-Léon Gérôme | Orientalism | NULL | NULL
The Grand Canal with San

In [155]:
# Pesquisando dados de estilos por inicio do nome
searchQuery = 'America'.lower()
qres = inferred_model.query(
    f"""
        PREFIX : <http://www.semanticweb.org/ericarfs/ontologies/2024/10/famouspaintings#>
        
        SELECT ?styleName
        WHERE {{
            ?style
                rdf:type :Style ;
                foaf:name ?styleName ;
            .
            FILTER (STRSTARTS(LCASE(STR(?styleName)), "{searchQuery}"))
        }}
    """,
    initNs={
        'rdf': rdf,
        'foaf': foaf,
    }
)

for row in qres:
    print(' | '.join([e or 'NULL' for e in row]))

American Landscape
American Art
America West
American West


In [156]:
# Pesquisando dados de temas por inicio do nome
searchQuery = 'A'.lower()
qres = inferred_model.query(
    f"""
        PREFIX : <http://www.semanticweb.org/ericarfs/ontologies/2024/10/famouspaintings#>
        
        SELECT ?subjectName
        WHERE {{
            ?subject
                rdf:type :Subject ;
                foaf:name ?subjectName ;
            .
            FILTER (STRSTARTS(LCASE(STR(?subjectName)), "{searchQuery}"))
        }}
    """,
    initNs={
        'rdf': rdf,
        'foaf': foaf,
    }
)

for row in qres:
    print(' | '.join([e or 'NULL' for e in row]))

Autumn/Fall
Abstract/Modern Art
Architectures


In [170]:
# Listar 10 obras de um museu junto com o estilo daquela obra
museumURL = 'http://www.clevelandart.org/'
qres = inferred_model.query(
    f"""
        PREFIX : <http://www.semanticweb.org/ericarfs/ontologies/2024/10/famouspaintings#>
        SELECT ?title ?style
        WHERE {{
            SERVICE <https://dbpedia.org/sparql> {{
                ?workDbo a dbo:Work ;
                         dbp:title ?title ;
                         dbo:museum/dbp:website <{museumURL}> .
            }}
            OPTIONAL {{
                ?workLocal
                    rdf:type :Work ;
                    foaf:name ?titleLocal ;
                    :hasStyle/foaf:name ?style ;
                .
                FILTER (STR(?title) = STR(?titleLocal))
            }}
        }}
        LIMIT 10
    """,
    initNs={
        'rdf': rdf,
        'foaf': foaf,
        'dbr': 'http://dbpedia.org/resource/',
        'dbo': 'http://dbpedia.org/ontology/',
        'dbp': 'http://dbpedia.org/property/',
    }
)

for row in qres:
    print(' | '.join([e or 'NULL' for e in row]))

Bottle, Glass, Fork | NULL
Stag at Sharkey's | NULL
Constellations | NULL
The Large Plane Trees | NULL
Cleveland Apollo | NULL
Fall of Simon Magus | NULL
The Crucifixion of Saint Andrew | Baroque
Un lièvre et un gigot de mouton | NULL
La Vie | NULL
Source | NULL


In [205]:
# Retornar os dados e a imagem de um artista pelo nome
artistName = 'Adélaïde Labille-Guiard'
qres = inferred_model.query(
    f"""
        PREFIX : <http://www.semanticweb.org/ericarfs/ontologies/2024/10/famouspaintings#>
        
        SELECT ?birth ?death ?style ?nationality ?image
        WHERE {{
            OPTIONAL {{
                SERVICE <https://dbpedia.org/sparql> {{
                    ?artistDbo a dbo:Animal ;
                        rdfs:label "{artistName}"@en ;
                        dbo:thumbnail ?image .
                }}
            }}
            ?artist
                rdf:type :Artist ;
                foaf:name "{artistName}" ;
            .
            OPTIONAL {{?artist :birth ?birth .}}
            OPTIONAL {{?artist :death ?death .}}
            OPTIONAL {{?artist :hasStyle/foaf:name ?style .}}
            OPTIONAL {{?artist :hasNationality/foaf:name ?nationality .}}
        }}
        LIMIT 1
    """,
    initNs={
        'rdf': rdf,
        'foaf': foaf,
        'dbr': 'http://dbpedia.org/resource/',
        'dbo': 'http://dbpedia.org/ontology/',
        'dbp': 'http://dbpedia.org/property/',
        'rdfs': 'http://www.w3.org/2000/01/rdf-schema#',
    }
)

for row in qres:
    print(' | '.join([e or 'NULL' for e in row]))

1749 | 1803 | Neoclassical | French | http://commons.wikimedia.org/wiki/Special:FilePath/Adélaïde_Labille-Guiard_-_Self-Portrait_with_Two_Pupils_-_The_Metropolitan_Museum_of_Art.jpg?width=300


In [222]:
# Retornar os dados e as coordenadas de um museu pelo nome
museumName = 'National Maritime Museum'
qres = inferred_model.query(
    f"""
        PREFIX : <http://www.semanticweb.org/ericarfs/ontologies/2024/10/famouspaintings#>
        
        SELECT ?country ?state ?city ?address ?postal ?phone ?url ?coords
        WHERE {{
            OPTIONAL {{
                SERVICE <https://dbpedia.org/sparql> {{
                    ?museumDbo a dbo:Building ;
                        rdfs:label "{museumName}"@en ;
                        georss:point ?coords .
                }}
            }}
            ?museum
                rdf:type :Museum ;
                foaf:name "{museumName}" ;
            .
            OPTIONAL {{?museum :hasCountry/foaf:name ?country .}}
            OPTIONAL {{?museum :state ?state .}}
            OPTIONAL {{?museum :city ?city .}}
            OPTIONAL {{?museum :address ?address .}}
            OPTIONAL {{?museum :postal ?postal .}}
            OPTIONAL {{?museum :phone ?phone .}}
            OPTIONAL {{?museum :url ?url .}}
        }}
        LIMIT 1
    """,
    initNs={
        'rdf': rdf,
        'foaf': foaf,
        'dbr': 'http://dbpedia.org/resource/',
        'dbo': 'http://dbpedia.org/ontology/',
        'dbp': 'http://dbpedia.org/property/',
        'rdfs': 'http://www.w3.org/2000/01/rdf-schema#',
        'georss': 'http://www.georss.org/georss/',
    }
)

for row in qres:
    print(' | '.join([e or 'NULL' for e in row]))

United Kingdom | SE10 9NF | London | Romney Rd | NULL | 44 20 8858 4422 | https://www.rmg.co.uk/national-maritime-museum | 51.481111 -0.005556
